In [50]:
import torch.nn as nn
import torch
import pandas as pd
from torch.utils.data import Dataset, DataLoader
import numpy as np
from tqdm.auto import tqdm
import torchmetrics
from sklearn.metrics import roc_auc_score
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score
from copy import copy
from sklearn.metrics import confusion_matrix
import torch.nn.functional as F
from tsai.all import GRU_FCN
import os
import argparse
from urllib.parse import urlparse
from io import StringIO
import json

device0 = torch.device("cpu")


class CustomDataset(Dataset):
    def __init__(self, data, transform=None, target_transform=None):
        self.PPG_data = data

    def __len__(self):
        return len(self.PPG_data)

    def __getitem__(self, idx):

        signal_tensor = self.PPG_data[idx]

        return signal_tensor


def test(model, dataloader):
    validation_tqdm = tqdm(dataloader)
    model.eval()
    val_count = 0
    predict_list = []
    with torch.no_grad():
        for x in validation_tqdm:
            val_count += 1
            x = x.to(device0).float()
            predict = model(x).to(device0)
            predict_list.append(F.sigmoid(predict).item())
    return predict_list


def interpolate_signal(signal, old_fs, new_fs):
    # Interpolate a signal from old_fs to new_fs using linear interpolation.
    # Create a time array for the original signal
    old_time = np.linspace(0, len(signal) / old_fs, len(signal), endpoint=False)

    # Create a time array for the resampled signal
    new_time = np.linspace(0, len(signal) / old_fs, int(len(signal) * new_fs / old_fs), endpoint=False)

    # Use numpy's interp function to perform the linear interpolation
    new_signal = np.interp(new_time, old_time, signal)

    return new_signal


In [86]:
json_path='response_1708317854685.json'
with open(json_path, 'r') as f:

    json_data = json.load(f)

# Now 'ppg_green' is a pandas DataFrame containing the CSV data
test_data=torch.empty(len(json_data), 1, 3000)
for i in range(len(json_data)):
    ppg_green=json_data[i]['ppgGreen'].split(',')
    ppg_green=np.array(ppg_green,dtype=np.float32)
    ppg_green=interpolate_signal(ppg_green, 25, 100)
    if len(ppg_green)>=3000:
        ppg_green=ppg_green[len(ppg_green)//2-1500:len(ppg_green)//2+1500]
    else:
        ppg_green=np.concatenate((ppg_green,np.zeros(3000-len(ppg_green))),axis=0)
    test_data[i,0,:]=torch.tensor(ppg_green)
        
        